In [2]:
## IMPORT PACKAGES

import os,gzip,itertools,csv,re,sys,io, Bio
import os.path
import urllib.request
import pandas as pd
import numpy as np
import urllib
import csv
import fastaparser as fp
import Bio
from Bio import SeqIO
from Bio import Entrez
from ftplib import FTP
import warnings
from pathlib import Path
import difflib 

warnings.filterwarnings("ignore", category=DeprecationWarning)

In [3]:
## READING IN SUMMARY DATA

dat = pd.read_csv('ensemble_metazoa.csv')


#with open('ensemble_metazoa.csv', "r", newline='') as csvfile:
#    read = csv.reader(csvfile, delimiter=',')
#    for row in read:
#        if row[2] != "Crustacea" or "Hemiptera" or "Diptera" or "Coleoptera" or "Hymenoptera" or "Lepidoptera" or "Chelicerata" or "Phthiraptera" or "Isoptera":
            
#dat
#dat.Classification

insecta = dat[(dat.Accession.isin(["GCA_000142985.2", "GCA_002204515.1", "GCA_001444175.1", "GCA_003254395.2", "GCA_000143395.2", "GCA_000775305.1", "GCA_000188095.2", "GCA_000214255.1", "GCA_000151625.1", "GCA_000648675.1", "GCA_000235995.2", "GCA_000313835.1", "GCA_000208615.1", "GCA_002892825.2", "GCA_003675905.1", "GCA_001187945.1", "GCA_000716385.1", "GCA_000371365.1", "GCA_009193385.2", "GCA_000006295.1", "GCA_000181055.3", "GCA_000828355.1", "GCA_000188075.2", "GCA_000611955.2", "GCA_001015335.1", "GCA_002237135.1", "GCA_000239435.1", "GCA_007210705.1", "GCA_000002335.3", "GCA_002443255.1", "GCA_000696155.1"]))]

insecta_df = insecta[["Name", "Classification", "Taxon ID", "Assembly", "Accession"]]

#removing duplicates
norm_insecta_df = insecta_df.drop(insecta_df.index[13])

#norm_insecta_df


In [4]:
## CODE TO PARSE FASTA FILES (Did not use this)
# this is code which will parse FASTA files
# define what a header looks like in FASTA format
def isheader(line):
    return line[0] == '>'

def aspairs(f):
    seq_id = ''
    sequence = ''
    for header,group in itertools.groupby(f, isheader):
        if header:
            line = next(group)
            seq_id = line[1:].split()[0]
        else:
            sequence = ''.join(line.strip() for line in group)
            yield seq_id, sequence
            
            
            

In [5]:
## GFF3 DATA PARSING

# Get names
GFFdirnames = os.listdir('GFF3s')

#Loop through files
GFFdir = os.scandir('GFF3s')
#print(GFFdirnames)

## Obtain genome stats
GFFdat = []
for f in GFFdir:
    with gzip.open(f, "rt") as fh:
        mygff = csv.reader(fh, delimiter="\t")
        count = 0
        gene_len = 0
        CDS_count = 0
        CDS_len = 0
        exon_count = 0
        exon_len = 0
        intron_count = 0
        intron_len = 0
        genome_size=0
        for row in mygff:
            if row[0].startswith("#"):
                continue
            #print(row)
            if row[2] == "gene":
                count += 1
                total_gene_len = int(row[4])-int(row[3])
                gene_len += total_gene_len
            if row[2] == "CDS":
                CDS_count += 1
                total_CDS_len =int(row[4])-int(row[3])
                CDS_len +=  total_CDS_len
            if row[2] == "exon":
                exon_count += 1
                total_exon_len =int(row[4])-int(row[3])
                exon_len +=  total_exon_len
                    
        mylist=list([Path(f).stem, count, gene_len, exon_count, exon_len, CDS_count, CDS_len])

    GFFdat.append(mylist)
#print(GFFdat)
           

In [6]:
## CREATE DF FROM GFF DATA

GFFdf = pd.DataFrame(GFFdat, columns = ["Name","Num_of_Genes","Gene_Length","Num_of_Exons","Exon_Length","Num_of_CDS","CDS_Length"]).sort_values("Name") 
GFFdf

#removing duplicates
norm1_GFFdf=GFFdf.drop(GFFdf.index[13])
norm_GFFdf=norm1_GFFdf.drop(GFFdf.index[27])

In [7]:
##OBTAIN GENOME SIZE

gensizedir = os.scandir('ncbi-genomes-2020-12-17')

names = []
size = []
for t in gensizedir:
    with open(t, "r") as txt:
        for x in txt:
            #print(x)
            if "Organism name" in x:
                names.append(x)
            if x.startswith('#'):
                continue
            elif 'Primary Assembly' in x:
                continue
            elif 'assembled-molecule' in x:
                continue
            elif 'Mitochondrion' in x:
                continue
            elif 'non-nuclear' in x:
                continue
            elif 'secondary' in x:
                continue
            elif 'Alts' in x:
                continue
            elif 'total-length' in x:
                size.append(x)

#Create array of genome size
gensize = np.array([706824083, 110781312, 621706043, 481803763, 650477627, 434128924, 389907520, 89583723, 248654244, 1870673364, 
    191142546, 317690795, 1776217663, 485009472, 249185056, 248577651, 750403944, 225251012, 971188624, 
    368925819, 398979080, 165928604, 297309692, 90815494, 2738704917, 117319042, 3088623987, 541675471, 56262437, 765382190])

#Create dict
d = {'Name': names,'Size (bp)':gensize}
#print(d)

#Create df
ncbidf= pd.DataFrame(d).sort_values('Name')

            

In [8]:
## Data Wrangling -- fixing some abnormal data...ended up being easier to create lists instead of merging dfs

Species = norm_insecta_df['Name'].to_list()
Order = norm_insecta_df['Classification'].to_list()
Genome_Size=ncbidf['Size (bp)'].to_list()
Num_of_Genes = norm_GFFdf['Num_of_Genes'].to_list()
Gene_Len = norm_GFFdf['Gene_Length'].to_list()
Num_of_Exons = norm_GFFdf['Num_of_Exons'].to_list()
Exon_Length = norm_GFFdf['Exon_Length'].to_list()
Num_of_CDS = norm_GFFdf['Num_of_CDS'].to_list()
CDS_Length = norm_GFFdf['CDS_Length'].to_list()


final_dict = {
                'Species': Species,
                'Order': Order,
                'Genome Size' : Genome_Size,
                'Num of Genes': Num_of_Genes,
                'Gene Length': Gene_Len,
                'Num of Exons': Num_of_Exons,
                'Exon Length': Exon_Length,
                'Num of CDS': Num_of_CDS,
                'CDS Length': Num_of_CDS
                    }


final_df=pd.DataFrame(final_dict)

final_df.to_csv('final_table.csv')

